In [1]:
import os
%pwd

'/Users/varundesai/Visual_Studio_Varun/Projects/Chicken-Disease-Classification/notebooks'

In [2]:
os.chdir("../")
%pwd

'/Users/varundesai/Visual_Studio_Varun/Projects/Chicken-Disease-Classification'

In [3]:
#Entity : entity is the return type of the function
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from src.chickenDiseaseClassification.constants import *
from src.chickenDiseaseClassification.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion 

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [7]:
import os
import urllib.request as request 
import zipfile
from chickenDiseaseClassification import logger
from chickenDiseaseClassification.utils.common import get_size

In [8]:
class DataIngestion:
    def __init__(
        self,
        config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        """
        Download the file from the source URL and save it locally
        """
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url= self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"Downloaded {filename} to {self.config.local_data_file} with following info: \n{headers}")
        else:
            logger.info(f"File {self.config.local_data_file} already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        """
        Unzip the downloaded file
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion = DataIngestion(config.get_data_ingestion_config())
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error(f"Error in DataIngestion: {e}")
    raise e

[2024-02-26 18:41:50,758: INFO: common: 31: yaml file: config/config.yaml is loaded successfully]
[2024-02-26 18:41:50,760: INFO: common: 31: yaml file: params.yaml is loaded successfully]
[2024-02-26 18:41:50,760: INFO: common: 58: Creating directory: artifacts]
[2024-02-26 18:41:50,761: INFO: common: 58: Creating directory: artifacts/data_ingestion]
[2024-02-26 18:41:57,136: INFO: 1212096454: 16: Downloaded artifacts/data_ingestion/data.zip to artifacts/data_ingestion/data.zip with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4AF8:38848C:159656:16EAEF:65DC8E17
Accept-Ranges: bytes
Date: Mon, 26 Feb 2024 13:11:52 GM